# Types Base - Foundation of Lionherd Type System

The `types.base` module provides the foundational building blocks for lionherd's type system:

**Core Components:**
- **Allowable Protocol**: Defines "allowed" keys/values for validation
- **Enum**: String-backed enums with Allowable implementation
- **ModelConfig**: Configuration for sentinel handling, validation, serialization
- **Params**: Frozen parameter objects with sentinel handling
- **DataClass**: Mutable dataclasses with validation
- **Meta**: Immutable metadata containers

**Key Features:**
- Sentinel values (Unset, Undefined, None, empty collections)
- Strict vs lenient validation modes
- Type-safe serialization
- Integration with Spec/Operable patterns

In [1]:
from dataclasses import dataclass, field

from lionherd_core.types import Unset
from lionherd_core.types.base import (
    DataClass,
    Enum,
    Meta,
    ModelConfig,
    Params,
)

## 1. The Allowable Protocol

Allowable is the foundation protocol defining what keys/values are permitted.

In [2]:
# Enum implements Allowable - returns allowed string values
class Status(Enum):
    PENDING = "pending"
    RUNNING = "running"
    COMPLETED = "completed"
    FAILED = "failed"


print(f"Allowed status values: {Status.allowed()}")
print(f"Type: {type(Status.allowed())}")

# Can use in validation
user_input = "running"
if user_input in Status.allowed():
    print(f"✓ '{user_input}' is valid")
else:
    print(f"✗ '{user_input}' is invalid")

Allowed status values: ('pending', 'running', 'completed', 'failed')
Type: <class 'tuple'>
✓ 'running' is valid


In [3]:
# Params implements Allowable - returns allowed parameter keys
@dataclass(slots=True, frozen=True, init=False)
class ModelParams(Params):
    model: str = Unset
    temperature: float = Unset
    max_tokens: int = Unset


print(f"Allowed parameter keys: {ModelParams.allowed()}")
print(f"Type: {type(ModelParams.allowed())}")

# Validation uses allowed() internally
params = ModelParams(model="gpt-4", temperature=0.7)
print(f"\nValid params created: {params}")

# Invalid key raises error
try:
    invalid = ModelParams(invalid_key="value")
except ValueError as e:
    print(f"\n✓ Invalid parameter rejected: {e}")

Allowed parameter keys: {'model', 'max_tokens', 'temperature'}
Type: <class 'set'>

Valid params created: ModelParams(model='gpt-4', temperature=0.7, max_tokens=Unset)

✓ Invalid parameter rejected: Invalid parameter: invalid_key


## 2. Enum - String-Backed Enumerations

Enum provides JSON-serializable string enums with Allowable protocol.

In [4]:
class Priority(Enum):
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"
    CRITICAL = "critical"


# Enum members are strings
p = Priority.HIGH
print(f"Value: {p} (type: {type(p)})")
print(f"Is string: {isinstance(p, str)}")

# Direct string operations
print(f"Uppercase: {p.upper()}")
print(f"Startswith 'h': {p.startswith('h')}")

# JSON serialization works out of the box
import json

data = {"priority": Priority.CRITICAL, "status": Status.RUNNING}
json_str = json.dumps(data)
print(f"\nJSON: {json_str}")
restored = json.loads(json_str)
print(f"Restored: {restored}")

Value: high (type: <enum 'Priority'>)
Is string: True
Uppercase: HIGH
Startswith 'h': True

JSON: {"priority": "critical", "status": "running"}
Restored: {'priority': 'critical', 'status': 'running'}


## 3. ModelConfig - Behavior Configuration

ModelConfig controls sentinel handling, validation, and serialization for Params/DataClass.

In [5]:
# Default config - lenient mode
default_config = ModelConfig()
print("Default configuration:")
print(f"  none_as_sentinel: {default_config.none_as_sentinel}")
print(f"  empty_as_sentinel: {default_config.empty_as_sentinel}")
print(f"  strict: {default_config.strict}")
print(f"  prefill_unset: {default_config.prefill_unset}")
print(f"  use_enum_values: {default_config.use_enum_values}")

Default configuration:
  none_as_sentinel: False
  empty_as_sentinel: False
  strict: False
  prefill_unset: True
  use_enum_values: False


In [6]:
# Strict config - requires all fields
@dataclass(slots=True, frozen=True, init=False)
class StrictParams(Params):
    _config = ModelConfig(strict=True)  # Requires all fields

    model: str = Unset
    temperature: float = Unset


# Missing required field raises ExceptionGroup
try:
    strict = StrictParams(model="gpt-4")  # Missing temperature
except ExceptionGroup as eg:
    print(f"✓ Strict validation caught {len(eg.exceptions)} errors:")
    for e in eg.exceptions:
        print(f"  - {e}")

# All fields provided - validates
strict = StrictParams(model="gpt-4", temperature=0.7)
print(f"\n✓ All fields provided: {strict}")

✓ Strict validation caught 1 errors:
  - Missing required parameter: temperature

✓ All fields provided: StrictParams(model='gpt-4', temperature=0.7)


In [7]:
# Sentinel configuration - controls what gets excluded from to_dict
@dataclass(slots=True, frozen=True, init=False)
class SentinelParams(Params):
    _config = ModelConfig(
        none_as_sentinel=True,  # Treat None as sentinel
        empty_as_sentinel=True,  # Treat [], {}, "" as sentinels
    )

    name: str | None = None
    tags: list = field(default_factory=list)
    metadata: dict = field(default_factory=dict)


params = SentinelParams(name=None, tags=[], metadata={})
print(f"to_dict excludes sentinels: {params.to_dict()}")

params2 = SentinelParams(name="test", tags=["a"], metadata={"k": "v"})
print(f"to_dict includes values: {params2.to_dict()}")

to_dict excludes sentinels: {}
to_dict includes values: {'tags': ['a'], 'metadata': {'k': 'v'}, 'name': 'test'}


## 4. Params - Frozen Parameter Objects

Params provides immutable parameter containers with sentinel handling and validation.

In [8]:
# Basic Params usage
@dataclass(slots=True, frozen=True, init=False)
class ChatParams(Params):
    model: str = Unset
    temperature: float = Unset
    max_tokens: int = Unset
    stream: bool = Unset


# Create with subset of parameters
params = ChatParams(model="gpt-4", temperature=0.7)
print(f"Created: {params}")
print(f"Allowed keys: {params.allowed()}")

# to_dict excludes Unset values
print(f"\nto_dict (excludes Unset): {params.to_dict()}")

Created: ChatParams(model='gpt-4', temperature=0.7, max_tokens=Unset, stream=Unset)
Allowed keys: {'model', 'max_tokens', 'temperature', 'stream'}

to_dict (excludes Unset): {'model': 'gpt-4', 'temperature': 0.7}


In [9]:
# Params is frozen - immutable
try:
    params.temperature = 0.8
except Exception as e:
    print(f"✓ Cannot modify frozen params: {type(e).__name__}")

# Use with_updates to create modified copy
updated = params.with_updates(temperature=0.8, stream=True)
print(f"\nOriginal: {params.to_dict()}")
print(f"Updated: {updated.to_dict()}")

✓ Cannot modify frozen params: FrozenInstanceError

Original: {'model': 'gpt-4', 'temperature': 0.7}
Updated: {'model': 'gpt-4', 'temperature': 0.8, 'stream': True}


In [10]:
# Params is hashable - safe for sets/dicts
p1 = ChatParams(model="gpt-4", temperature=0.7)
p2 = ChatParams(model="gpt-4", temperature=0.7)
p3 = ChatParams(model="gpt-3.5", temperature=0.7)

print(f"p1 == p2: {p1 == p2}")
print(f"p1 == p3: {p1 == p3}")
print(f"hash(p1) == hash(p2): {hash(p1) == hash(p2)}")

# Works in sets
param_set = {p1, p2, p3}
print(f"\nSet size (p1 and p2 deduplicated): {len(param_set)}")

p1 == p2: True
p1 == p3: False
hash(p1) == hash(p2): True

Set size (p1 and p2 deduplicated): 2


In [11]:
# Container copying - shallow vs deep
@dataclass(slots=True, frozen=True, init=False)
class ContainerParams(Params):
    tags: list = field(default_factory=list)
    config: dict = field(default_factory=dict)


params = ContainerParams(tags=["a", "b"], config={"key": [1, 2, 3]})

# No copy - shares containers (dangerous!)
no_copy = params.with_updates()
print(f"No copy shares containers: {params.tags is no_copy.tags}")

# Shallow copy - copies top-level containers
shallow = params.with_updates(copy_containers="shallow")
print(f"Shallow copy: {params.tags is shallow.tags}")
print(f"But nested still shared: {params.config['key'] is shallow.config['key']}")

# Deep copy - fully independent
deep = params.with_updates(copy_containers="deep")
print(f"Deep copy independent: {params.config['key'] is deep.config['key']}")

No copy shares containers: True
Shallow copy: False
But nested still shared: True
Deep copy independent: False


## 5. DataClass - Mutable Validated Dataclasses

DataClass provides mutable dataclasses with the same validation patterns as Params.

In [12]:
# DataClass is mutable (@dataclass(unsafe_hash=True) preserves parent's hash)
from dataclasses import dataclass


@dataclass(unsafe_hash=True)
class Task(DataClass):
    title: str = Unset
    status: str = Unset
    priority: str = Unset
    tags: list = field(default_factory=list)


task = Task(title="Implement feature", status="pending", priority="high")
print(f"Created: {task.title} [{task.status}]")

# Can modify fields
task.status = "running"
task.tags.append("backend")
print(f"Modified: {task.title} [{task.status}] tags={task.tags}")

Created: Implement feature [pending]
Modified: Implement feature [running] tags=['backend']


In [13]:
# DataClass also supports strict mode
@dataclass(unsafe_hash=True)
class StrictTask(DataClass):
    _config = ModelConfig(strict=True)

    title: str = Unset
    status: str = Unset


try:
    strict_task = StrictTask(title="Test")  # Missing status
except ExceptionGroup as eg:
    print(f"✓ Strict DataClass validation: {eg.exceptions[0]}")

strict_task = StrictTask(title="Test", status="pending")
print(f"✓ All fields provided: {strict_task}")

✓ Strict DataClass validation: Missing required parameter: status
✓ All fields provided: StrictTask(title='Test', status='pending')


In [14]:
# DataClass supports equality
t1 = Task(title="Task 1", status="pending", priority="high")
t2 = Task(title="Task 1", status="pending", priority="high")
t3 = Task(title="Task 2", status="pending", priority="high")

print(f"t1 == t2 (same values): {t1 == t2}")
print(f"t1 == t3 (different title): {t1 == t3}")

# Note: DataClass with mutable fields (like list) cannot be hashed
# For hashable dataclasses, use immutable field types or Params instead

t1 == t2 (same values): True
t1 == t3 (different title): False


## 6. Sentinel Values - Unset vs Undefined

Lionherd uses three sentinel values for different purposes:
- **Unset**: Field has no value (default, excluded from serialization)
- **Undefined**: Field was never initialized (detected by prefill_unset)
- **None**: Explicit null value (included in serialization unless none_as_sentinel=True)

In [15]:
# Unset vs None distinction
@dataclass(slots=True, frozen=True, init=False)
class OptionParams(Params):
    name: str | None = Unset
    value: int | None = Unset


# Unset field - excluded from to_dict
p1 = OptionParams(name="test")
print(f"Unset excluded: {p1.to_dict()}")

# None value - included in to_dict
p2 = OptionParams(name="test", value=None)
print(f"None included: {p2.to_dict()}")

Unset excluded: {'name': 'test'}
None included: {'value': None, 'name': 'test'}


In [16]:
# Prefill unset - detects Undefined fields
@dataclass(unsafe_hash=True)
class PrefillTask(DataClass):
    _config = ModelConfig(prefill_unset=True)  # Default is True

    title: str  # No default - will be Undefined
    status: str = "pending"  # Has default


# Without __post_init__, title would be Undefined
# With prefill_unset=True, Undefined fields get Unset
task = PrefillTask(title="Test")
print(f"Status after prefill: {task.status}")

# Check internal state
task2 = PrefillTask(title="Test")
print(
    f"Status is Unset: {task2.status is Unset}"  # Will be False - has default value
)

Status after prefill: pending
Status is Unset: False


In [17]:
# Empty collections as sentinels
@dataclass(slots=True, frozen=True, init=False)
class EmptyParams(Params):
    _config = ModelConfig(empty_as_sentinel=True)

    tags: list = field(default_factory=list)
    metadata: dict = field(default_factory=dict)
    name: str = ""


# Empty collections excluded when empty_as_sentinel=True
p1 = EmptyParams(tags=[], metadata={}, name="")
print(f"Empty values excluded: {p1.to_dict()}")

# Non-empty included
p2 = EmptyParams(tags=["a"], metadata={"k": "v"}, name="test")
print(f"Non-empty included: {p2.to_dict()}")

Empty values excluded: {}
Non-empty included: {'tags': ['a'], 'name': 'test', 'metadata': {'k': 'v'}}


## 7. Enum Value Normalization

ModelConfig controls whether enums serialize as enum objects or their string values.

In [18]:
# Default - enums stay as enum objects
@dataclass(slots=True, frozen=True, init=False)
class DefaultEnumParams(Params):
    status: Status = Unset
    priority: Priority = Unset


params = DefaultEnumParams(status=Status.RUNNING, priority=Priority.HIGH)
data = params.to_dict()
print(f"Default (enum objects): {data}")
print(f"Status type: {type(data['status'])}")

Default (enum objects): {'priority': <Priority.HIGH: 'high'>, 'status': <Status.RUNNING: 'running'>}
Status type: <enum 'Status'>


In [19]:
# use_enum_values=True - convert to strings
@dataclass(slots=True, frozen=True, init=False)
class StringEnumParams(Params):
    _config = ModelConfig(use_enum_values=True)

    status: Status = Unset
    priority: Priority = Unset


params = StringEnumParams(status=Status.RUNNING, priority=Priority.HIGH)
data = params.to_dict()
print(f"use_enum_values=True (strings): {data}")
print(f"Status type: {type(data['status'])}")

# Now JSON-serializable without custom encoder
import json

json_str = json.dumps(data)
print(f"JSON: {json_str}")

use_enum_values=True (strings): {'priority': 'high', 'status': 'running'}
Status type: <class 'str'>
JSON: {"priority": "high", "status": "running"}


## 8. Meta - Immutable Metadata Containers

Meta provides hashable key-value pairs for metadata, with special handling for callables.

In [20]:
# Basic metadata
m1 = Meta(key="version", value="1.0.0")
m2 = Meta(key="author", value="Ocean")

print(f"m1: {m1}")
print(f"m2: {m2}")

# Meta is hashable - safe for sets
metadata_set = {m1, m2}
print(f"\nMetadata set: {metadata_set}")

m1: Meta(key='version', value='1.0.0')
m2: Meta(key='author', value='Ocean')

Metadata set: {Meta(key='author', value='Ocean'), Meta(key='version', value='1.0.0')}


In [21]:
# Callable values - compared by identity
def func1():
    pass


def func2():
    pass


m1 = Meta(key="validator", value=func1)
m2 = Meta(key="validator", value=func1)  # Same function object
m3 = Meta(key="validator", value=func2)  # Different function

print(f"m1 == m2 (same function): {m1 == m2}")
print(f"m1 == m3 (different function): {m1 == m3}")

# Hash by identity for callables
print(f"\nhash(m1) == hash(m2): {hash(m1) == hash(m2)}")
print(f"hash(m1) == hash(m3): {hash(m1) == hash(m3)}")

m1 == m2 (same function): True
m1 == m3 (different function): False

hash(m1) == hash(m2): True
hash(m1) == hash(m3): False


In [22]:
# Unhashable values - fallback to string representation
m_list = Meta(key="data", value=[1, 2, 3])
m_dict = Meta(key="config", value={"a": 1, "b": 2})

print(f"List metadata: {m_list}")
print(f"Dict metadata: {m_dict}")

# Can hash (uses string fallback)
print(f"\nCan hash list metadata: {hash(m_list)}")
print(f"Can hash dict metadata: {hash(m_dict)}")

List metadata: Meta(key='data', value=[1, 2, 3])
Dict metadata: Meta(key='config', value={'a': 1, 'b': 2})

Can hash list metadata: 5357622055777332727
Can hash dict metadata: 2432044317472346102


## 9. Integration with Spec/Operable Patterns

The base types enable structured validation and type-safe operations throughout lionherd.

In [23]:
# Example: LLM parameters with validation
@dataclass(slots=True, frozen=True, init=False)
class LLMParams(Params):
    _config = ModelConfig(strict=False, use_enum_values=True)

    model: str = Unset
    temperature: float = Unset
    max_tokens: int = Unset
    top_p: float = Unset
    frequency_penalty: float = Unset

    def _validate(self) -> None:
        """Custom validation logic."""
        Params._validate(self)  # Call parent validation

        errors = []

        # Temperature range
        if self.temperature is not Unset and not (0.0 <= self.temperature <= 2.0):
            errors.append(ValueError(f"temperature must be 0-2, got {self.temperature}"))

        # max_tokens positive
        if self.max_tokens is not Unset and self.max_tokens <= 0:
            errors.append(ValueError(f"max_tokens must be positive, got {self.max_tokens}"))

        # top_p range
        if self.top_p is not Unset and not (0.0 <= self.top_p <= 1.0):
            errors.append(ValueError(f"top_p must be 0-1, got {self.top_p}"))

        if errors:
            raise ExceptionGroup("Validation errors", errors)


# Valid parameters
params = LLMParams(model="gpt-4", temperature=0.7, max_tokens=100)
print(f"✓ Valid params: {params.to_dict()}")

# Invalid temperature
try:
    invalid = LLMParams(model="gpt-4", temperature=3.0)
except ExceptionGroup as eg:
    print(f"\n✓ Validation caught error: {eg.exceptions[0]}")

✓ Valid params: {'max_tokens': 100, 'model': 'gpt-4', 'temperature': 0.7}

✓ Validation caught error: temperature must be 0-2, got 3.0


In [24]:
# Example: Type-safe enum usage
class ModelProvider(Enum):
    OPENAI = "openai"
    ANTHROPIC = "anthropic"
    GOOGLE = "google"


@dataclass(slots=True, frozen=True, init=False)
class ServiceParams(Params):
    _config = ModelConfig(strict=True, use_enum_values=True)

    provider: ModelProvider = Unset
    model: str = Unset
    api_key: str = Unset


# Type-safe creation
params = ServiceParams(provider=ModelProvider.OPENAI, model="gpt-4", api_key="sk-...")

# Serializes to strings
data = params.to_dict()
print(f"Serialized: {data}")
print(f"Provider is string: {isinstance(data['provider'], str)}")

# Validation uses allowed() from enum
print(f"\nAllowed providers: {ModelProvider.allowed()}")

Serialized: {'provider': 'openai', 'model': 'gpt-4', 'api_key': 'sk-...'}
Provider is string: True

Allowed providers: ('openai', 'anthropic', 'google')


In [25]:
# Example: default_kw() for function parameter passing
@dataclass(slots=True, frozen=True, init=False)
class RequestParams(Params):
    url: str = Unset
    method: str = Unset
    headers: dict = field(default_factory=dict)
    timeout: int = Unset
    kwargs: dict = field(default_factory=dict)  # Extra kwargs


params = RequestParams(
    url="https://api.example.com",
    method="POST",
    timeout=30,
    kwargs={"verify": False, "proxies": {"http": "proxy.example.com"}},
)

# default_kw() flattens kwargs into top-level dict
kw = params.default_kw()
print(f"Flattened kwargs: {kw}")
print(f"verify in kw: {'verify' in kw}")
print(f"kwargs in kw: {'kwargs' in kw}")

Flattened kwargs: {'method': 'POST', 'timeout': 30, 'url': 'https://api.example.com', 'verify': False, 'proxies': {'http': 'proxy.example.com'}}
verify in kw: True
kwargs in kw: False


## 10. Practical Patterns

Common patterns combining these base types.

In [26]:
# Pattern 1: Strict API parameters with validation
class HTTPMethod(Enum):
    GET = "GET"
    POST = "POST"
    PUT = "PUT"
    DELETE = "DELETE"


@dataclass(slots=True, frozen=True, init=False)
class APIRequest(Params):
    _config = ModelConfig(strict=True, use_enum_values=True)

    method: HTTPMethod = Unset
    endpoint: str = Unset
    headers: dict = field(default_factory=dict)
    body: dict | None = None

    def _validate(self) -> None:
        Params._validate(self)  # Call parent validation

        errors = []

        # GET/DELETE shouldn't have body
        if self.method in (HTTPMethod.GET, HTTPMethod.DELETE) and self.body:
            errors.append(ValueError(f"{self.method} requests should not have body"))

        # Endpoint must start with /
        if not self.endpoint.startswith("/"):
            errors.append(ValueError(f"Endpoint must start with /, got {self.endpoint}"))

        if errors:
            raise ExceptionGroup("API request validation errors", errors)


# Valid POST request
request = APIRequest(
    method=HTTPMethod.POST,
    endpoint="/api/users",
    headers={"Content-Type": "application/json"},
    body={"name": "Alice"},
)
print(f"✓ Valid POST: {request.to_dict()}")

# Invalid GET with body
try:
    invalid = APIRequest(method=HTTPMethod.GET, endpoint="/api/users", body={"x": 1})
except ExceptionGroup as eg:
    print(f"\n✓ Validation error: {eg.exceptions[0]}")

✓ Valid POST: {'endpoint': '/api/users', 'method': 'POST', 'body': {'name': 'Alice'}, 'headers': {'Content-Type': 'application/json'}}

✓ Validation error: Missing required parameter: headers


In [27]:
# Pattern 2: Configuration with defaults and overrides
@dataclass(unsafe_hash=True)
class AppConfig(DataClass):
    """Mutable application configuration."""

    # Core settings
    environment: str = "development"
    debug: bool = True
    log_level: str = "INFO"

    # Database
    db_host: str = "localhost"
    db_port: int = 5432
    db_name: str = "app"

    # Features
    features: list = field(default_factory=list)


# Default config
config = AppConfig()
print(f"Default config: {config.to_dict()}")

# Production override
config.environment = "production"
config.debug = False
config.log_level = "WARNING"
config.db_host = "prod.db.example.com"
print(f"\nProduction config: {config.to_dict()}")

# Can add features dynamically
config.features.extend(["auth", "analytics"])
print(f"\nWith features: {config.features}")

Default config: {'db_host': 'localhost', 'debug': True, 'db_port': 5432, 'features': [], 'log_level': 'INFO', 'environment': 'development', 'db_name': 'app'}

Production config: {'db_host': 'prod.db.example.com', 'debug': False, 'db_port': 5432, 'features': [], 'log_level': 'WARNING', 'environment': 'production', 'db_name': 'app'}

With features: ['auth', 'analytics']


In [28]:
# Pattern 3: Builder pattern with Params
@dataclass(slots=True, frozen=True, init=False)
class QueryParams(Params):
    """SQL query parameters."""

    table: str = Unset
    columns: list = field(default_factory=list)
    where: dict = field(default_factory=dict)
    order_by: list = field(default_factory=list)
    limit: int | None = None
    offset: int | None = None


# Build query step by step
query = QueryParams(table="users", columns=["id", "name", "email"])
print(f"Base query: {query.to_dict()}")

# Add WHERE clause
query = query.with_updates(where={"status": "active", "age__gte": 18})
print(f"\nWith WHERE: {query.to_dict()}")

# Add ordering and pagination
query = query.with_updates(
    order_by=["created_at DESC"], limit=10, offset=0, copy_containers="shallow"
)
print(f"\nWith ORDER/LIMIT: {query.to_dict()}")

Base query: {'table': 'users', 'columns': ['id', 'name', 'email']}

With WHERE: {'where': {'status': 'active', 'age__gte': 18}, 'table': 'users', 'columns': ['id', 'name', 'email']}

With ORDER/LIMIT: {'where': {'status': 'active', 'age__gte': 18}, 'order_by': ['created_at DESC'], 'table': 'users', 'columns': ['id', 'name', 'email'], 'limit': 10, 'offset': 0}


## Summary Checklist

**Allowable Protocol:**
- ✅ Defines `allowed()` method returning allowed keys/values
- ✅ Implemented by Enum (values), Params (keys), DataClass (keys)
- ✅ Foundation for validation throughout lionherd

**Enum:**
- ✅ String-backed enums (Python 3.11+)
- ✅ JSON-serializable out of the box
- ✅ Implements Allowable for validation

**ModelConfig:**
- ✅ Controls sentinel handling (none_as_sentinel, empty_as_sentinel)
- ✅ Controls validation (strict, prefill_unset)
- ✅ Controls serialization (use_enum_values)

**Params:**
- ✅ Frozen (immutable) parameter objects
- ✅ Sentinel-aware serialization via to_dict()
- ✅ Hashable and equality support
- ✅ with_updates() for functional updates
- ✅ Container copying (shallow/deep)
- ✅ default_kw() for function parameter passing

**DataClass:**
- ✅ Mutable validated dataclasses
- ✅ Same validation patterns as Params
- ✅ Hashable and equality support
- ✅ Useful for configuration and state

**Sentinel Values:**
- ✅ Unset: No value (excluded from serialization)
- ✅ Undefined: Never initialized (detected by prefill_unset)
- ✅ None: Explicit null (included unless none_as_sentinel=True)
- ✅ Empty collections: Excluded when empty_as_sentinel=True

**Meta:**
- ✅ Immutable key-value pairs
- ✅ Hashable (special handling for callables)
- ✅ Callable values compared by identity
- ✅ Unhashable values fallback to string hash

**Integration Patterns:**
- ✅ Type-safe validation with custom _validate()
- ✅ Enum-based type safety
- ✅ Builder pattern with with_updates()
- ✅ Configuration management
- ✅ API parameter validation

**Next Steps:**
- See `Spec` for output schema definition
- See `Operable` for structured operations
- See `Element` for entity base classes using these patterns